# Task 5 of sheet 4
Numerical Methods 2, WS 2021/22<br>
by Clemens Wager

(2 Points) Consider a randomly generated matrix A elem R (15x30) and compute its SVD (take a
package/built-in function). Compute the rank-10 (low rank) matrix A_10 from the SVD of A by
discarding the smallest three singular values sigma_11,...,sigma_15. Calculate the error in the Frobenius norm ||A-A_10||_F and sqrt(sum(j=11 to 15)(sigma^2_j)).

In [1]:
import numpy as np
import scipy.sparse as sparse
import scipy.linalg as sla

#### Generate a sparse reproducible 15x30 matrix 

In [2]:
# generate symmetric matrix S
m = 15
n = 30
density = 0.3
print(f"m={m}; n={n}")

# ATTENTION random.seed() from random library does not work with other libraries!
np.random.seed(42)

# Let U be a matrix of uniformly distributed random numbers.
# type(A)=coo_matrix # very fast conversion to and from CSR/CSC formats
A = sparse.random(m,n, density=density)
print(f"A.shape = {A.shape}")

A_coo = A.copy()
#print("type(A_coo) =",type(A_coo))
#print(f"Matrix A (coo) =\n{A_coo}\n")

A_arr = A.toarray()
#print("type(A_arr) =",type(A_arr))
#print(f"Matrix A (array) =\n{A_arr}\n")

# Rank of A
print(f"Rank of original A = {np.linalg.matrix_rank(A_arr)}")

# Source: https://cmdlinetips.com/2019/02/how-to-create-random-sparse-matrix-of-specific-density/

m=15; n=30
A.shape = (15, 30)
Rank of original A = 15


#### Compute the SVD

In [3]:
U, s, Vh = sla.svd(a=A_arr) # package function for SVD

In [4]:
# U: Unitary matrix having left singular vectors as columns.
print(f"U {U.shape}")
#print(f"{U}")

# s: The singular values, sorted in decreasing order. 
print(f"s {s.shape}")
print(f"{s}")

# Vh: Unitary matrix having right singular vectors as rows.
print(f"Vh {Vh.shape}")
#print(f"{Vh}")

U (15, 15)
s (15,)
[4.00174009 2.43791807 2.19375267 2.03231234 1.81298175 1.61032585
 1.56733038 1.48280019 1.40771779 1.17278639 1.05562052 0.85826203
 0.82703156 0.74706396 0.6873167 ]
Vh (30, 30)


#### Check whether the SVD worked as intended

In [5]:
# Reconstruct original A with full rank
# define a sigma mxn with all singular values along diagonal
sigma_full = np.zeros((m,n))
for i in range(min(m, n)):
    sigma_full[i, i] = s[i]

A_prod = np.dot( np.dot(U,sigma_full), Vh)
print(f"A_prod {A_prod.shape}")
print(f"rank(A_prod) = {np.linalg.matrix_rank(A_prod)}")
# Close result?
print(f"A_prod close to A_arr: {np.allclose(A_arr, A_prod)}")

A_prod (15, 30)
rank(A_prod) = 15
A_prod close to A_arr: True


#### Compute rank-10 (low rank) matrix A10 from the SVD  

In [6]:
# ...by discarding the smallest ("THREE") FIVE singular values
s_dis = s.copy()
s_dis[-5:] = 0
s_dis_shape = s_dis.shape
print(f"s_dis {s_dis_shape}")
print(f"{s_dis}\n")

# define a simga mxn with reduced singular values along diagonal
sigma_red = np.zeros((m,n))
for i in range(min(m, n)):
    sigma_red[i, i] = s_dis[i]
    
print(f"sigma_red.shape {sigma_red.shape}\n")
#print(f"{sigma_red}\n")
print(f"np.diag(sigma_red)\n{np.diag(sigma_red)}\n")

# Reconstruct A with reduced rank
A10 = np.dot( np.dot(U,sigma_red), Vh)
print(f"A10 {A10.shape}")
print(f"rank(A10) = {np.linalg.matrix_rank(A10)}")

s_dis (15,)
[4.00174009 2.43791807 2.19375267 2.03231234 1.81298175 1.61032585
 1.56733038 1.48280019 1.40771779 1.17278639 0.         0.
 0.         0.         0.        ]

sigma_red.shape (15, 30)

np.diag(sigma_red)
[4.00174009 2.43791807 2.19375267 2.03231234 1.81298175 1.61032585
 1.56733038 1.48280019 1.40771779 1.17278639 0.         0.
 0.         0.         0.        ]

A10 (15, 30)
rank(A10) = 10


#### Calculate errors

In [11]:
# calculate 1) the error (Frobenius norm)
ferr = np.linalg.norm(A_arr-A10, ord='fro')
print(f"->Error in the Frobenius norm =\t{ferr}")

# 2) the error resulting from the discarded singular values
discerr=0
for sv in s[-5:]:
    discerr += sv**2
discerr = np.sqrt(discerr)
print(f"->Error calculated from \nthe discarded singular values =\t{discerr}")
print(f"->Difference in errors = {abs(ferr-discerr)}")

->Error in the Frobenius norm =	1.8882368538057257
->Error calculated from 
the discarded singular values =	1.8882368538057253
->Difference in errors = 4.440892098500626e-16
